# Intellectual Property Data Integration Guide

This notebook documents integrations with public APIs for gathering intellectual property data from official sources including USPTO, WIPO, Google Patents, and other resources.

## Available IP Data APIs

### 1. **USPTO APIs** (United States Patent and Trademark Office)
- **Patent Search API**: Free access to patent data
- **Trademark Search API**: Trademark registration data
- **Patent Examination Data**: Real-time prosecution history
- Base URL: `https://developer.uspto.gov/`

### 2. **WIPO APIs** (World Intellectual Property Organization)
- **Global Patent Index (PATENTSCOPE)**: International patent data
- **Global Brand Database**: International trademark data
- Base URL: `https://www.wipo.int/`

### 3. **Google Patents API**
- Patent search and analysis
- Citation data
- Related patents
- Base URL: `https://patents.google.com/`

### 4. **EPO (European Patent Office)**
- **Espacenet API**: European and international patent data
- **Global Patent Index**: Linked data access
- Base URL: `https://www.epo.org/`

### 5. **JPIP (Japanese Patent Information Platform)**
- Japanese patent data
- Base URL: `https://jpplatform.inpit.go.jp/`

---

## Setup and Requirements

Install required packages for API integration:

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime
import time
from typing import Dict, List, Optional
from urllib.parse import urlencode, quote
import xml.etree.ElementTree as ET

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Master Setup - Initialize All IP APIs (Run This First!)

Run this single cell to initialize all IP database APIs and the consolidated search interface. You only need to run this once per session.

In [5]:
# ============================================================================
# MASTER SETUP - Initialize All IP APIs
# ============================================================================
# Run this cell once to initialize all IP database APIs
# Note: USPTO Patent API is temporarily disabled (waiting for API key)

# Import required dependencies
from typing import Dict, List, Optional
from urllib.parse import urlencode, quote
import pandas as pd

print("Initializing IP APIs...")
print("=" * 80)

# 1. Google Patents API
class GooglePatentsAPI:
    """Interface for Google Patents search"""
    
    def __init__(self):
        self.base_url = "https://patents.google.com"
    
    def search_patents(self, query: str) -> str:
        """Generate search URL for Google Patents"""
        params = {"q": query, "status": "GRANT", "sort": "new"}
        query_string = urlencode(params)
        return f"{self.base_url}/?{query_string}"
    
    def search_by_assignee(self, assignee: str) -> str:
        """Search by patent assignee"""
        return f"{self.base_url}/?assignee={quote(assignee)}&status=GRANT"
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        return f"{self.base_url}/?inventor={quote(inventor)}&status=GRANT"

# 2. WIPO PatentScope API
class WIPOPatentScopeAPI:
    """Interface for WIPO PATENTSCOPE API"""
    
    def __init__(self):
        self.search_url = "https://patentscope.wipo.int/search/en/search.jsf"
    
    def construct_search_url(self, query: str) -> str:
        """Construct PATENTSCOPE search URL"""
        params = {"query": query, "type": "text"}
        query_string = urlencode(params)
        return f"{self.search_url}?{query_string}"
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant name"""
        return f"{self.search_url}?query=PA:{quote(applicant)}"
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        return f"{self.search_url}?query=IN:{quote(inventor)}"
    
    def get_global_brand_search_url(self, brand: str) -> str:
        """Search Global Brand Database"""
        return "https://www.wipo.int/branddb/en/"

# 3. EPO Espacenet API
class EPOEspacenetAPI:
    """Interface for EPO Espacenet patent search"""
    
    def __init__(self):
        self.base_url = "https://worldwide.espacenet.com"
    
    def construct_search_url(self, query: str) -> str:
        """Construct Espacenet search URL"""
        return f"{self.base_url}/advancedSearch?txt={quote(query)}&st=advanced&pr=WO"
    
    def search_simple(self, keywords: str) -> str:
        """Simple keyword search"""
        return f"{self.base_url}/advancedSearch?txt={quote(keywords)}&st=advanced&pr=WO"
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant name"""
        return f"{self.base_url}/advancedSearch?an={quote(applicant)}&st=advanced&pr=WO"
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        return f"{self.base_url}/advancedSearch?in={quote(inventor)}&st=advanced&pr=WO"
    
    def search_by_ipc(self, ipc_code: str) -> str:
        """Search by IPC classification"""
        return f"{self.base_url}/advancedSearch?ic={quote(ipc_code)}&st=advanced&pr=WO"

# 4. USPTO Trademark API
class USPTOTrademarkAPI:
    """Interface for USPTO Trademark Search API"""
    
    def __init__(self):
        self.base_url = "https://tmsearch.uspto.gov/bins/gate.pl"
    
    def search_by_mark(self, mark: str) -> str:
        """Search by trademark mark/name"""
        return f"{self.base_url}?f=toc&rs=on&Query={quote(mark)}&action=goto_toc"
    
    def search_by_owner(self, owner: str) -> str:
        """Search by owner/applicant name"""
        return f"{self.base_url}?f=toc&rs=on&Query=OWNER%3A{quote(owner)}&action=goto_toc"

# 5. KIPO APIs
class KIPOPatentAPI:
    """Interface for KIPO Patent Search API"""
    
    def __init__(self):
        self.patent_search_url = "https://www.kipris.or.kr/cipf/jsp/patent/searchResult.jsp"
    
    def search_by_title(self, title: str) -> str:
        """Search by patent title/keywords"""
        return f"{self.patent_search_url}?Query={quote(title)}&Query_Type=title"
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant/assignee"""
        return f"{self.patent_search_url}?Query={quote(applicant)}&Query_Type=applicant"
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        return f"{self.patent_search_url}?Query={quote(inventor)}&Query_Type=inventor"

class KIPOTrademarkAPI:
    """Interface for KIPO Trademark Search API"""
    
    def __init__(self):
        self.tm_search = "https://www.kipris.or.kr/cipf/jsp/tm/selectTmTotalList.jsp"
    
    def search_by_mark_name(self, mark: str) -> str:
        """Search by trademark mark name"""
        return f"{self.tm_search}?searchKey={quote(mark)}&searchKeyType=mark_name"

class KIPODesignAPI:
    """Interface for KIPO Design Search API"""
    
    def __init__(self):
        self.base_url = "https://www.kipris.or.kr/cipf/jsp/design/selectDesignTotalList.jsp"
    
    def search_by_title(self, title: str) -> str:
        """Search design registrations by title"""
        return f"{self.base_url}?searchKey={quote(title)}&searchKeyType=title"

# 6. Consolidated Search Interface
class ConsolidatedIPSearch:
    """Unified interface for searching across multiple IP databases"""
    
    def __init__(self):
        self.google_patents = GooglePatentsAPI()
        self.wipo = WIPOPatentScopeAPI()
        self.epo = EPOEspacenetAPI()
        self.uspto_trademarks = USPTOTrademarkAPI()
        self.kipo_patent = KIPOPatentAPI()
        self.kipo_trademark = KIPOTrademarkAPI()
        self.kipo_design = KIPODesignAPI()
    
    def search_patent_globally(self, query: str, search_type: str = "keyword") -> Dict[str, str]:
        """Search for patents across multiple databases"""
        results = {
            "Google Patents": None,
            "WIPO PATENTSCOPE": None,
            "EPO Espacenet": None,
            "KIPO": None
        }
        
        if search_type == "keyword":
            results["Google Patents"] = self.google_patents.search_patents(query)
            results["WIPO PATENTSCOPE"] = self.wipo.construct_search_url(query)
            results["EPO Espacenet"] = self.epo.search_simple(query)
            results["KIPO"] = self.kipo_patent.search_by_title(query)
        elif search_type == "applicant":
            results["Google Patents"] = self.google_patents.search_by_assignee(query)
            results["WIPO PATENTSCOPE"] = self.wipo.search_by_applicant(query)
            results["EPO Espacenet"] = self.epo.search_by_applicant(query)
            results["KIPO"] = self.kipo_patent.search_by_applicant(query)
        elif search_type == "inventor":
            results["Google Patents"] = self.google_patents.search_by_inventor(query)
            results["WIPO PATENTSCOPE"] = self.wipo.search_by_inventor(query)
            results["EPO Espacenet"] = self.epo.search_by_inventor(query)
            results["KIPO"] = self.kipo_patent.search_by_inventor(query)
        
        return results
    
    def search_trademarks(self, mark: str, regions: List[str] = None) -> Dict[str, str]:
        """Search for trademarks across multiple regions"""
        if regions is None:
            regions = ["US", "International", "Korea"]
        
        results = {}
        if "US" in regions:
            results["USPTO"] = self.uspto_trademarks.search_by_mark(mark)
        if "International" in regions:
            results["WIPO Global Brand"] = self.wipo.get_global_brand_search_url(mark)
        if "Korea" in regions:
            results["KIPO"] = self.kipo_trademark.search_by_mark_name(mark)
        
        return results
    
    def create_search_report(self, query: str, search_type: str = "keyword") -> pd.DataFrame:
        """Create a report with search links across databases"""
        results = self.search_patent_globally(query, search_type)
        report_data = []
        for database, url in results.items():
            if url:
                report_data.append({
                    "Database": database,
                    "Search Type": search_type.capitalize(),
                    "Query": query,
                    "URL": url
                })
        return pd.DataFrame(report_data)

# Initialize all API clients
print("✓ Initializing Google Patents API")
google_patents = GooglePatentsAPI()

print("✓ Initializing WIPO PATENTSCOPE API")
wipo_api = WIPOPatentScopeAPI()

print("✓ Initializing EPO Espacenet API")
epo_api = EPOEspacenetAPI()

print("✓ Initializing USPTO Trademark API")
uspto_tm = USPTOTrademarkAPI()

print("✓ Initializing KIPO Patent API")
kipo_patent = KIPOPatentAPI()

print("✓ Initializing KIPO Trademark API")
kipo_trademark = KIPOTrademarkAPI()

print("✓ Initializing KIPO Design API")
kipo_design = KIPODesignAPI()

print("✓ Initializing Consolidated IP Search Interface")
ip_search = ConsolidatedIPSearch()

print("=" * 80)
print("✅ All IP APIs initialized successfully!")
print("=" * 80)
print("\nYou can now use the following objects to search IP databases:")
print("  • ip_search.search_patent_globally(query, search_type)")
print("  • ip_search.search_trademarks(mark, regions)")
print("  • ip_search.create_search_report(query, search_type)")
print("\nOr access individual APIs:")
print("  • google_patents, wipo_api, epo_api")
print("  • uspto_tm, kipo_patent, kipo_trademark, kipo_design")
print("\n📝 Note: USPTO Patent API is awaiting API key registration.")
print("   Once available, it will be added back for patent searches.")
print("\nRun the example cells below to see how to use them!")

Initializing IP APIs...
✓ Initializing Google Patents API
✓ Initializing WIPO PATENTSCOPE API
✓ Initializing EPO Espacenet API
✓ Initializing USPTO Trademark API
✓ Initializing KIPO Patent API
✓ Initializing KIPO Trademark API
✓ Initializing KIPO Design API
✓ Initializing Consolidated IP Search Interface
✅ All IP APIs initialized successfully!

You can now use the following objects to search IP databases:
  • ip_search.search_patent_globally(query, search_type)
  • ip_search.search_trademarks(mark, regions)
  • ip_search.create_search_report(query, search_type)

Or access individual APIs:
  • google_patents, wipo_api, epo_api
  • uspto_tm, kipo_patent, kipo_trademark, kipo_design

📝 Note: USPTO Patent API is awaiting API key registration.
   Once available, it will be added back for patent searches.

Run the example cells below to see how to use them!


---

## 1. USPTO Patent API Integration

### USPTO API Key
First, obtain an API key from the USPTO developer portal:
- Navigate to https://developer.uspto.gov/
- Register for an account
- Request an API key

In [ ]:
class USPTOPatentAPI:
    """Interface for USPTO Patent Search API"""
    
    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key
        self.base_url = "https://api.patentsview.org/patents/query"
        self.headers = {"Content-Type": "application/json"}
    
    def search_by_applicant(self, applicant_name: str, limit: int = 50) -> Dict:
        """Search patents by applicant name"""
        params = {
            "q": {"assignee_organization": applicant_name},
            "f": ["patent_number", "patent_title", "patent_date", "assignee_organization"],
            "o": {"page": 1, "per_page": limit}
        }
        
        try:
            response = requests.post(self.base_url, json=params, headers=self.headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error searching patents: {e}")
            return {"patents": []}
    
    def search_by_inventor(self, inventor_name: str, limit: int = 50) -> Dict:
        """Search patents by inventor name"""
        params = {
            "q": {"inventor_name": inventor_name},
            "f": ["patent_number", "patent_title", "patent_date", "inventor_name"],
            "o": {"page": 1, "per_page": limit}
        }
        
        try:
            response = requests.post(self.base_url, json=params, headers=self.headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error searching patents: {e}")
            return {"patents": []}
    
    def search_by_keyword(self, keywords: str, limit: int = 50) -> Dict:
        """Search patents by keywords in title/abstract"""
        params = {
            "q": {
                "_or": [
                    {"patent_title": keywords},
                    {"patent_abstract": keywords}
                ]
            },
            "f": ["patent_number", "patent_title", "patent_date", "patent_abstract"],
            "o": {"page": 1, "per_page": limit}
        }
        
        try:
            response = requests.post(self.base_url, json=params, headers=self.headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error searching patents: {e}")
            return {"patents": []}
    
    def search_by_cpc_classification(self, cpc_code: str, limit: int = 50) -> Dict:
        """Search patents by CPC classification code"""
        params = {
            "q": {"cpc_classification_title": cpc_code},
            "f": ["patent_number", "patent_title", "patent_date", "cpc_classification_title"],
            "o": {"page": 1, "per_page": limit}
        }
        
        try:
            response = requests.post(self.base_url, json=params, headers=self.headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error searching patents: {e}")
            return {"patents": []}
    
    def convert_to_dataframe(self, results: Dict) -> pd.DataFrame:
        """Convert API results to pandas DataFrame"""
        patents = results.get("patents", [])
        
        flat_data = []
        for patent in patents:
            flat_data.append({
                'Patent Number': patent.get('patent_number'),
                'Title': patent.get('patent_title'),
                'Date': patent.get('patent_date'),
                'Assignee': patent.get('assignee_organization'),
                'Inventors': ', '.join([inv.get('inventor_name', '') for inv in patent.get('inventor_name', [])]),
                'Abstract': patent.get('patent_abstract', '')[:200] + '...' if patent.get('patent_abstract') else ''
            })
        
        return pd.DataFrame(flat_data)

# Initialize USPTO API client
uspto_api = USPTOPatentAPI()
print("USPTO Patent API client initialized successfully!")

### Example: Search for Patents by Applicant

In [ ]:
# Example: Search patents for a specific applicant
results = uspto_api.search_by_applicant("Apple Inc", limit=10)
if results.get("patents"):
    df_patents = uspto_api.convert_to_dataframe(results)
    print(f"Found {len(df_patents)} patents for Apple Inc")
    df_patents.head()
else:
    print("No patents found")

---

## 2. Google Patents API Integration

Google Patents provides free patent search without requiring authentication. Use web scraping or their public API endpoints.

In [ ]:
class GooglePatentsAPI:
    """Interface for Google Patents search"""
    
    def __init__(self):
        self.base_url = "https://patents.google.com"
        self.search_url = f"{self.base_url}/xhr/query"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            "Accept": "application/json"
        }
    
    def construct_search_url(self, query: str, search_type: str = "q") -> str:
        """Construct a search URL for Google Patents"""
        # URL format: https://patents.google.com/?q=query&status=GRANT&oq=query
        params = {
            search_type: query,
            "status": "GRANT",
            "sort": "new"
        }
        query_string = urlencode(params)
        return f"{self.base_url}/?{query_string}"
    
    def search_patents(self, query: str, filters: Optional[Dict] = None) -> str:
        """Generate search URL for Google Patents"""
        # Note: Google Patents doesn't have a public JSON API anymore
        # This returns the URL to visit or scrape
        url = self.construct_search_url(query)
        return url
    
    def search_by_assignee(self, assignee: str) -> str:
        """Search by patent assignee"""
        url = f"{self.base_url}/?assignee={quote(assignee)}&status=GRANT"
        return url
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        url = f"{self.base_url}/?inventor={quote(inventor)}&status=GRANT"
        return url
    
    def search_by_classification(self, cpc: str) -> str:
        """Search by CPC classification"""
        url = f"{self.base_url}/?CPC={quote(cpc)}&status=GRANT"
        return url
    
    def construct_patent_url(self, patent_number: str) -> str:
        """Construct direct URL to a specific patent"""
        # Format: US20210234567A1
        return f"{self.base_url}/patent/{patent_number}/"

google_patents = GooglePatentsAPI()

# Example searches
print("Google Patents Search URLs:")
print(f"\n1. Search by keyword:")
print(google_patents.search_patents("machine learning"))

print(f"\n2. Search by assignee:")
print(google_patents.search_by_assignee("Microsoft"))

print(f"\n3. Search by inventor:")
print(google_patents.search_by_inventor("John Smith"))

print(f"\n4. Direct patent URL:")
print(google_patents.construct_patent_url("US10123456B1"))

---

## 3. WIPO PatentScope API Integration

The World Intellectual Property Organization (WIPO) offers access to international patent data through PATENTSCOPE.

In [ ]:
class WIPOPatentScopeAPI:
    """Interface for WIPO PATENTSCOPE API"""
    
    def __init__(self):
        self.base_url = "https://www.wipo.int/edocs/en"
        self.search_url = "https://patentscope.wipo.int/search/en/search.jsf"
        self.api_url = "https://patentscope.wipo.int/ajax"
        
    def construct_search_url(self, query: str, search_type: str = "text") -> str:
        """Construct PATENTSCOPE search URL"""
        params = {
            "query": query,
            "type": search_type,  # 'text', 'number', 'inventor', etc.
        }
        query_string = urlencode(params)
        return f"{self.search_url}?{query_string}"
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant name"""
        url = f"{self.search_url}?query=PA:{quote(applicant)}"
        return url
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        url = f"{self.search_url}?query=IN:{quote(inventor)}"
        return url
    
    def search_by_classification(self, ipc: str) -> str:
        """Search by IPC classification"""
        url = f"{self.search_url}?query=IC:{quote(ipc)}"
        return url
    
    def construct_patent_url(self, publication_number: str) -> str:
        """Construct direct URL to PATENTSCOPE entry"""
        # Publication number format: WO2020/123456
        return f"https://patentscope.wipo.int/search/en/detail.jsf?docId={publication_number}"
    
    def get_global_brand_search_url(self, brand: str) -> str:
        """Search Global Brand Database"""
        # Global Brand Database: https://www.wipo.int/branddb/
        url = f"https://www.wipo.int/branddb/en/"
        return url

wipo_api = WIPOPatentScopeAPI()

print("WIPO PatentScope Search URLs:")
print(f"\n1. Applicant search:")
print(wipo_api.search_by_applicant("Philips"))

print(f"\n2. Inventor search:")
print(wipo_api.search_by_inventor("Albert Einstein"))

print(f"\n3. IPC Classification search:")
print(wipo_api.search_by_classification("H04L"))

print(f"\n4. Patent detail URL:")
print(wipo_api.construct_patent_url("WO2021123456"))

---

## 4. EPO (European Patent Office) API Integration

The European Patent Office provides Espacenet for patent search and the Global Patent Index with linked data.

In [ ]:
class EPOEspacenetAPI:
    """Interface for EPO Espacenet patent search"""
    
    def __init__(self):
        self.base_url = "https://worldwide.espacenet.com"
        self.linked_data_url = "https://data.epo.org"
        
    def construct_search_url(self, query: str, search_field: str = "txt") -> str:
        """Construct Espacenet search URL"""
        # search_field: txt (keywords), ic (IPC), cpc (CPC), an (applicant), in (inventor), pn (publication number)
        params = {
            search_field: query,
            "st": "advanced",  # advanced search
            "pr": "WO",  # All countries
        }
        query_string = urlencode(params)
        return f"{self.base_url}/advancedSearch?{query_string}"
    
    def search_simple(self, keywords: str) -> str:
        """Simple keyword search"""
        url = f"{self.base_url}/advancedSearch?txt={quote(keywords)}&st=advanced&pr=WO"
        return url
    
    def search_by_publication_number(self, pub_number: str) -> str:
        """Search by publication number"""
        url = f"{self.base_url}/advancedSearch?pn={quote(pub_number)}&st=advanced"
        return url
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant name"""
        url = f"{self.base_url}/advancedSearch?an={quote(applicant)}&st=advanced&pr=WO"
        return url
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name"""
        url = f"{self.base_url}/advancedSearch?in={quote(inventor)}&st=advanced&pr=WO"
        return url
    
    def search_by_ipc(self, ipc_code: str) -> str:
        """Search by IPC classification"""
        url = f"{self.base_url}/advancedSearch?ic={quote(ipc_code)}&st=advanced&pr=WO"
        return url
    
    def search_by_cpc(self, cpc_code: str) -> str:
        """Search by CPC classification"""
        url = f"{self.base_url}/advancedSearch?cpc={quote(cpc_code)}&st=advanced&pr=WO"
        return url
    
    def get_patent_url(self, patent_number: str, country_code: str = "WO") -> str:
        """Get direct URL to patent document"""
        return f"{self.base_url}/textdoc?DB=EPODOC&AN={country_code}{patent_number}&F=0&QPN={country_code}{patent_number}"
    
    def get_linked_data_url(self, patent_number: str) -> str:
        """Get linked data URL for patent"""
        # EPO linked data provides RDF/XML format
        return f"{self.linked_data_url}/patent/{patent_number}"

epo_api = EPOEspacenetAPI()

print("EPO Espacenet Search URLs:")
print(f"\n1. Simple keyword search:")
print(epo_api.search_simple("artificial intelligence"))

print(f"\n2. Applicant search:")
print(epo_api.search_by_applicant("SIEMENS"))

print(f"\n3. IPC classification search:")
print(epo_api.search_by_ipc("G06F"))

print(f"\n4. Patent detail page:")
print(epo_api.get_patent_url("EP3000000B1"))

---

## 5. USPTO Trademark API Integration

Search and retrieve trademark registration data from the USPTO.

In [ ]:
class USPTOTrademarkAPI:
    """Interface for USPTO Trademark Search API"""
    
    def __init__(self):
        self.base_url = "https://tmsearch.uspto.gov/bins/gate.pl"
        self.api_url = "https://tsdr.uspto.gov/api/v1"
        
    def construct_search_url(self, query: str, search_type: str = "text") -> str:
        """Construct trademark search URL"""
        # search_type: text, owner, designer, correspondent, etc.
        params = {
            "f": "toc",
            "rs": True,
            "Query": query,
            "action": "goto_toc"
        }
        query_string = urlencode(params)
        return f"{self.base_url}?{query_string}"
    
    def search_by_mark(self, mark: str) -> str:
        """Search by trademark mark/name"""
        url = f"{self.base_url}?f=toc&rs=on&Query={quote(mark)}&action=goto_toc"
        return url
    
    def search_by_owner(self, owner: str) -> str:
        """Search by owner/applicant name"""
        url = f"{self.base_url}?f=toc&rs=on&Query=OWNER%3A{quote(owner)}&action=goto_toc"
        return url
    
    def search_by_status(self, status: str) -> str:
        """Search by registration status"""
        # status: REGISTERED, ABANDONED, CANCELLED, PENDING, etc.
        url = f"{self.base_url}?f=toc&rs=on&Query=STATUS%3A{quote(status)}&action=goto_toc"
        return url
    
    def get_registration_details_url(self, serial_number: str) -> str:
        """Get TSDR (Trademark Status & Document Retrieval) URL"""
        return f"https://tsdr.uspto.gov/cgi-bin/statusmp?sn={serial_number}"
    
    def get_api_registration_data(self, serial_number: str) -> Dict:
        """Fetch detailed trademark data via API"""
        url = f"{self.api_url}/trademarks/{serial_number}"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching trademark data: {e}")
            return {}

uspto_tm = USPTOTrademarkAPI()

print("USPTO Trademark Search URLs:")
print(f"\n1. Search by mark name:")
print(uspto_tm.search_by_mark("APPLE"))

print(f"\n2. Search by owner:")
print(uspto_tm.search_by_owner("Microsoft"))

print(f"\n3. Search by registration status:")
print(uspto_tm.search_by_status("REGISTERED"))

print(f"\n4. TSDR details page:")
print(uspto_tm.get_registration_details_url("87123456"))

---

## 6. Utility Class: Consolidated IP Search Interface

Create a unified interface to search across multiple IP databases:

In [ ]:
class ConsolidatedIPSearch:
    """Unified interface for searching across multiple IP databases"""
    
    def __init__(self):
        self.uspto_patents = USPTOPatentAPI()
        self.google_patents = GooglePatentsAPI()
        self.wipo = WIPOPatentScopeAPI()
        self.epo = EPOEspacenetAPI()
        self.uspto_trademarks = USPTOTrademarkAPI()
        self.kipo_patent = KIPOPatentAPI()
        self.kipo_trademark = KIPOTrademarkAPI()
        self.kipo_design = KIPODesignAPI()
    
    def search_patent_globally(self, query: str, search_type: str = "keyword") -> Dict[str, str]:
        """Search for patents across multiple databases"""
        results = {
            "USPTO": None,
            "Google Patents": None,
            "WIPO PATENTSCOPE": None,
            "EPO Espacenet": None,
            "KIPO": None
        }
        
        if search_type == "keyword":
            results["USPTO"] = "USPTO PatentsView Query" 
            results["Google Patents"] = self.google_patents.search_patents(query)
            results["WIPO PATENTSCOPE"] = self.wipo.construct_search_url(query)
            results["EPO Espacenet"] = self.epo.search_simple(query)
            results["KIPO"] = self.kipo_patent.search_by_title(query)
        
        elif search_type == "applicant":
            results["USPTO"] = "USPTO Applicant Search"
            results["Google Patents"] = self.google_patents.search_by_assignee(query)
            results["WIPO PATENTSCOPE"] = self.wipo.search_by_applicant(query)
            results["EPO Espacenet"] = self.epo.search_by_applicant(query)
            results["KIPO"] = self.kipo_patent.search_by_applicant(query)
        
        elif search_type == "inventor":
            results["USPTO"] = "USPTO Inventor Search"
            results["Google Patents"] = self.google_patents.search_by_inventor(query)
            results["WIPO PATENTSCOPE"] = self.wipo.search_by_inventor(query)
            results["EPO Espacenet"] = self.epo.search_by_inventor(query)
            results["KIPO"] = self.kipo_patent.search_by_inventor(query)
        
        return results
    
    def search_trademarks(self, mark: str, regions: List[str] = None) -> Dict[str, str]:
        """Search for trademarks across multiple regions"""
        if regions is None:
            regions = ["US", "International", "Korea"]
        
        results = {}
        
        if "US" in regions:
            results["USPTO"] = self.uspto_trademarks.search_by_mark(mark)
        
        if "International" in regions:
            results["WIPO Global Brand"] = self.wipo.get_global_brand_search_url(mark)
        
        if "Korea" in regions:
            results["KIPO"] = self.kipo_trademark.search_by_mark_name(mark)
        
        return results
    
    def create_search_report(self, query: str, search_type: str = "keyword") -> pd.DataFrame:
        """Create a report with search links across databases"""
        results = self.search_patent_globally(query, search_type)
        
        report_data = []
        for database, url in results.items():
            if url:
                report_data.append({
                    "Database": database,
                    "Search Type": search_type.capitalize(),
                    "Query": query,
                    "URL": url
                })
        
        return pd.DataFrame(report_data)

# Initialize consolidated search
ip_search = ConsolidatedIPSearch()

# Example: Create a comprehensive search report
print("=" * 80)
print("CONSOLIDATED IP SEARCH REPORT")
print("=" * 80)

search_query = "blockchain"
report = ip_search.create_search_report(search_query, "keyword")
print(f"\nSearch Query: '{search_query}'")
print(report.to_string(index=False))

---

## 7. API Reference and Documentation

### Key Resources

| API | Endpoint | Authentication | Status | Documentation |
|-----|----------|-----------------|--------|---|
| **Google Patents** | `patents.google.com` | None (Free) | ✅ Active | https://patents.google.com/ |
| **WIPO PATENTSCOPE** | `patentscope.wipo.int` | None (Free) | ✅ Active | https://www.wipo.int/patentscope/ |
| **EPO Espacenet** | `worldwide.espacenet.com` | None (Free) | ✅ Active | https://worldwide.espacenet.com/ |
| **USPTO Trademarks** | `tmsearch.uspto.gov` | None (Free) | ✅ Active | https://www.uspto.gov/trademarks |
| **KIPO Patents** | `kipris.or.kr` | Free Registration | ✅ Active | https://www.kipris.or.kr/ |
| **KIPO Trademarks** | `kipris.or.kr` | Free Registration | ✅ Active | https://www.kipo.go.kr/ |
| **KIPO Designs** | `kipris.or.kr` | Free Registration | ✅ Active | https://www.kipo.go.kr/ |
| **USPTO Patents** | `api.patentsview.org` | API Key Required | ⏳ Pending | https://developer.uspto.gov/ |

### Classification Systems

- **IPC (International Patent Classification)**: Global standard for patent classification
- **CPC (Cooperative Patent Classification)**: Joint classification system (USPTO + EPO)
- **USPC (United States Patent Classification)**: Legacy USPTO classification (being phased out)
- **Nice Classification**: Used for trademark classification

### Common Search Parameters

```
- Patent Number: EP1234567, US10123456B1, WO2020/123456, KR10123456789
- Applicant/Assignee: Company name or organization
- Inventor: Individual name
- Keywords: Title and abstract search terms
- Classification: IPC, CPC, or USPC codes
- Date Range: Publication or filing date ranges
- Status: GRANTED, PENDING, ABANDONED, etc.
```

---

## 8. Best Practices for IP Data Gathering

### Rate Limiting & Respectful Scraping
- **Always check robots.txt** before scraping
- **Respect API rate limits** and quota restrictions
- **Add delays** between requests (2-5 seconds)
- **Use official APIs** when available rather than scraping
- **Cache results** to minimize repeated requests

### Data Quality
- **Validate patent numbers** before searching
- **Handle multiple inventors/assignees** appropriately
- **Normalize company names** (exact names vs. abbreviations)
- **Verify publication dates** (filing vs. publication vs. issue date)
- **Check patent status** (active, expired, abandoned)

### Compliance
- **Review Terms of Service** for each platform
- **Respect robots.txt** and metadata directives
- **Don't overload servers** with excessive requests
- **Include proper attribution** when sharing data
- **Understand data licensing** (public domain varies by jurisdiction)

### Data Storage & Analysis
- **Store structured data** (JSON, CSV, or database)
- **Maintain data currency** with periodic updates
- **Track data source and retrieval date**
- **Create indexes** on commonly searched fields
- **Document data provenance** and transformations

---

## 9. KIPO (Korean Patent Office) API Integration

The Korea Intellectual Property Office (KIPO) provides comprehensive APIs for patent, trademark, and design data retrieval.

### KIPO API Resources
- **Patent Search API**: Domestic and international patent data
- **Trademark Search API**: Korean trademark registrations
- **Design Search API**: Design registrations
- **Official Documentation**: https://www.kipo.go.kr/en/
- **API Key Required**: Yes, registration required

In [ ]:
class KIPOPatentAPI:
    """Interface for KIPO (Korea Patent Office) Patent Search API"""
    
    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key
        self.base_url = "https://www.kipris.or.kr/cipf/jsp/patent/searchResult.jsp"
        self.patent_search_url = "https://www.kipris.or.kr/cipf/jsp/patent/searchResult.jsp"
        self.api_base = "https://www.kipris.or.kr/openapi"
        
    def construct_search_url(self, query: str, search_type: str = "title") -> str:
        """Construct KIPRIS search URL"""
        # search_type: title, inventor, applicant, abstract, cpc
        params = {
            "Query": query,
            "Query_Type": search_type,
            "numPerPage": 50
        }
        query_string = urlencode(params)
        return f"{self.patent_search_url}?{query_string}"
    
    def search_by_title(self, title: str) -> str:
        """Search by patent title/keywords"""
        url = f"{self.patent_search_url}?Query={quote(title)}&Query_Type=title"
        return url
    
    def search_by_inventor(self, inventor: str) -> str:
        """Search by inventor name (Korean name support)"""
        url = f"{self.patent_search_url}?Query={quote(inventor)}&Query_Type=inventor"
        return url
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search by applicant/assignee"""
        url = f"{self.patent_search_url}?Query={quote(applicant)}&Query_Type=applicant"
        return url
    
    def search_by_publication_number(self, pub_number: str) -> str:
        """Search by publication number (KR format: KR10XXXXXXXX)"""
        url = f"{self.patent_search_url}?Query={quote(pub_number)}&Query_Type=keyword"
        return url
    
    def search_by_cpc(self, cpc_code: str) -> str:
        """Search by CPC classification"""
        url = f"{self.patent_search_url}?Query={quote(cpc_code)}&Query_Type=cpc"
        return url
    
    def get_patent_detail_url(self, publication_number: str) -> str:
        """Get detailed patent information URL"""
        # Publication numbers: KR10XXXXXXXX (utility patent), KR20XXXXXXXX (design)
        return f"https://www.kipris.or.kr/cipf/jsp/patent/patentDetail.jsp?publNo={publication_number}"
    
    def search_domestic_international(self, query: str, search_type: str = "title", 
                                     domestic_only: bool = False) -> str:
        """Search domestic and international patents"""
        url = self.search_by_title(query) if search_type == "title" else self.search_by_applicant(query)
        if domestic_only:
            url += "&country_code=KR"
        return url

class KIPOTrademarkAPI:
    """Interface for KIPO Trademark Search API"""
    
    def __init__(self):
        self.base_url = "https://www.kipris.or.kr/cipf/jsp/tm/selectTmTotalList.jsp"
        self.tm_search = "https://www.kipris.or.kr/cipf/jsp/tm/selectTmTotalList.jsp"
        
    def construct_search_url(self, mark: str, search_type: str = "mark_name") -> str:
        """Construct trademark search URL"""
        params = {
            "searchKey": mark,
            "searchKeyType": search_type,
            "pageNum": 1,
            "pageSize": 50
        }
        query_string = urlencode(params)
        return f"{self.tm_search}?{query_string}"
    
    def search_by_mark_name(self, mark: str) -> str:
        """Search by trademark mark name"""
        url = f"{self.tm_search}?searchKey={quote(mark)}&searchKeyType=mark_name"
        return url
    
    def search_by_owner(self, owner: str) -> str:
        """Search by trademark owner"""
        url = f"{self.tm_search}?searchKey={quote(owner)}&searchKeyType=owner"
        return url
    
    def get_trademark_detail_url(self, application_number: str) -> str:
        """Get detailed trademark information"""
        return f"https://www.kipris.or.kr/cipf/jsp/tm/tmDetail.jsp?applNo={application_number}"

class KIPODesignAPI:
    """Interface for KIPO Design Search API"""
    
    def __init__(self):
        self.base_url = "https://www.kipris.or.kr/cipf/jsp/design/selectDesignTotalList.jsp"
        
    def search_by_title(self, title: str) -> str:
        """Search design registrations by title"""
        url = f"{self.base_url}?searchKey={quote(title)}&searchKeyType=title"
        return url
    
    def search_by_applicant(self, applicant: str) -> str:
        """Search design registrations by applicant"""
        url = f"{self.base_url}?searchKey={quote(applicant)}&searchKeyType=applicant"
        return url
    
    def get_design_detail_url(self, registration_number: str) -> str:
        """Get detailed design information"""
        return f"https://www.kipris.or.kr/cipf/jsp/design/designDetail.jsp?regNo={registration_number}"

# Initialize KIPO APIs
kipo_patent = KIPOPatentAPI()
kipo_trademark = KIPOTrademarkAPI()
kipo_design = KIPODesignAPI()

print("KIPO API clients initialized successfully!")
print("\nKIPRIS (Korean Patent Information Service) provides:")
print("- Patent search (domestic Korean patents)")
print("- Trademark search (Korean trademark registrations)")
print("- Design search (Korean design registrations)")
print("- Published PCT applications visible in Korea")

### Example: KIPO Patent Search

In [ ]:
# Example KIPO searches
print("=" * 80)
print("KIPO PATENT SEARCH EXAMPLES")
print("=" * 80)

print("\n1. Search by patent title (keyword):")
print(kipo_patent.search_by_title("배터리 기술"))  # Battery technology

print("\n2. Search by applicant:")
print(kipo_patent.search_by_applicant("삼성"))  # Samsung

print("\n3. Search by CPC classification:")
print(kipo_patent.search_by_cpc("H01M"))  # Battery/cell technology

print("\n4. Get patent detail page:")
print(kipo_patent.get_patent_detail_url("KR10123456789"))

print("\n" + "=" * 80)
print("KIPO TRADEMARK SEARCH EXAMPLES")
print("=" * 80)

print("\n1. Search trademark by mark name:")
print(kipo_trademark.search_by_mark_name("Galaxy"))

print("\n2. Search trademark by owner:")
print(kipo_trademark.search_by_owner("LG"))

print("\n3. Get trademark detail page:")
print(kipo_trademark.get_trademark_detail_url("4000123456"))

print("\n" + "=" * 80)
print("KIPO DESIGN SEARCH EXAMPLES")
print("=" * 80)

print("\n1. Search design by title:")
print(kipo_design.search_by_title("스마트폰"))  # Smartphone

print("\n2. Search design by applicant:")
print(kipo_design.search_by_applicant("현대자동차"))  # Hyundai Motor

---

## 10. How to Perform Consolidated Searches

**QUICK START**: Run the **Master Setup cell** above (right after the imports) once to initialize all IP APIs, then use the examples below.

The `ConsolidatedIPSearch` class allows you to search across multiple IP databases simultaneously. Here are practical examples:

### Example 1: Search Patents by Keyword Across All Databases

In [ ]:
# Search for patents related to "quantum computing" across all databases
print("=" * 100)
print("CONSOLIDATED PATENT SEARCH - KEYWORD: 'QUANTUM COMPUTING'")
print("=" * 100)

search_results = ip_search.search_patent_globally("quantum computing", search_type="keyword")

print("\nAvailable search links across databases:\n")
for database, url in search_results.items():
    print(f"{database}:")
    print(f"  {url}\n")

# Generate a formatted report
report = ip_search.create_search_report("quantum computing", "keyword")
print("\nFormatted Search Report:")
print(report.to_string(index=False))

### Example 2: Search Patents by Applicant/Company

In [ ]:
# Find all patents by a specific company across multiple databases
print("=" * 100)
print("CONSOLIDATED PATENT SEARCH - APPLICANT: '{company_query}'")
print("=" * 100)

company_query = input("Enter company name to search for patents: ")
company_results = ip_search.search_patent_globally(company_query, search_type="applicant")

print(f"\nPatent search links for {company_query}:\n")
for database, url in company_results.items():
    print(f"{database}:")
    print(f"  {url}\n")

# Create report
company_report = ip_search.create_search_report("Tesla", "applicant")
print("\nCompany Patent Report:")
print(company_report.to_string(index=False))

CONSOLIDATED PATENT SEARCH - APPLICANT: 'TESLA'

Patent search links for Masimo Corporation:

Google Patents:
  https://patents.google.com/?assignee=Masimo%20Corporation&status=GRANT

WIPO PATENTSCOPE:
  https://patentscope.wipo.int/search/en/search.jsf?query=PA:Masimo%20Corporation

EPO Espacenet:
  https://worldwide.espacenet.com/advancedSearch?an=Masimo%20Corporation&st=advanced&pr=WO

KIPO:
  https://www.kipris.or.kr/cipf/jsp/patent/searchResult.jsp?Query=Masimo%20Corporation&Query_Type=applicant


Company Patent Report:
        Database Search Type Query                                                                                        URL
  Google Patents   Applicant Tesla                                    https://patents.google.com/?assignee=Tesla&status=GRANT
WIPO PATENTSCOPE   Applicant Tesla                           https://patentscope.wipo.int/search/en/search.jsf?query=PA:Tesla
   EPO Espacenet   Applicant Tesla                  https://worldwide.espacenet.com/advance

### Example 3: Search Trademarks Across Multiple Regions

In [ ]:
# Search for trademark registrations across US, International, and Korean markets
print("=" * 100)
print("CONSOLIDATED TRADEMARK SEARCH - MARK: 'INNOVATE'")
print("=" * 100)

# Search in all regions (default behavior)
trademark_results = ip_search.search_trademarks("Innovate")

print("\nTrademark search links across regions:\n")
for region, url in trademark_results.items():
    print(f"{region}:")
    print(f"  {url}\n")

# Search only in specific regions
print("\n" + "-" * 100)
print("TRADEMARK SEARCH - US ONLY")
print("-" * 100)

us_only_results = ip_search.search_trademarks("Innovate", regions=["US"])
for region, url in us_only_results.items():
    print(f"\n{region}:")
    print(f"  {url}")

### Example 4: Direct API Access for Specific Databases

In [ ]:
# You can also access individual API classes for more targeted searches
print("=" * 100)
print("DIRECT API ACCESS - INDIVIDUAL DATABASE SEARCHES")
print("=" * 100)

# EPO Espacenet - Search by IPC classification for battery technology
print("\n1. EPO - IPC Classification Search (H01M - Battery/Cell Technology):")
epo_url = epo_api.search_by_ipc("H01M")
print(f"   {epo_url}")

# Google Patents - Search by assignee
print("\n2. Google Patents - Search by Assignee (Toyota):")
google_url = google_patents.search_by_assignee("Toyota")
print(f"   {google_url}")

# WIPO - Search by inventor
print("\n3. WIPO PATENTSCOPE - Search by Inventor (Bill Gates):")
wipo_url = wipo_api.search_by_inventor("Bill Gates")
print(f"   {wipo_url}")

# KIPO - Search by applicant (Korean company)
print("\n4. KIPO - Search by Applicant (LG Electronics - Korean):")
kipo_url = kipo_patent.search_by_applicant("LG")
print(f"   {kipo_url}")

# USPTO - Get specific patent detail page
print("\n5. USPTO Patents - Direct Patent Number Search:")
uspto_patent_url = "Visit: https://patents.google.com/patent/US10123456B1"
print(f"   {uspto_patent_url}")

### Example 5: Building Custom Search Workflows

In [ ]:
# Custom search workflow - Compare patents across multiple jurisdictions
def compare_patent_coverage(query: str, search_type: str = "keyword") -> pd.DataFrame:
    """
    Search for patents in a specific jurisdiction and get a comparison report
    """
    results = []
    
    # Search across all databases
    global_results = ip_search.search_patent_globally(query, search_type)
    
    for database, url in global_results.items():
        results.append({
            "Jurisdiction": database,
            "Search Type": search_type.capitalize(),
            "Query": query,
            "Search URL": url,
            "Ready to Search": "✓" if url else "✗"
        })
    
    return pd.DataFrame(results)

# Example: Compare coverage for "AI Patent Search"
print("=" * 100)
print("GLOBAL PATENT COVERAGE ANALYSIS - QUERY: 'ARTIFICIAL INTELLIGENCE'")
print("=" * 100)

coverage_report = compare_patent_coverage("artificial intelligence", "keyword")
print("\n" + coverage_report.to_string(index=False))

print("\n" + "=" * 100)
print("QUICK START GUIDE")
print("=" * 100)

quick_guide = """
To perform a consolidated search:

1. KEYWORD SEARCH (find patents about a technology):
   results = ip_search.search_patent_globally("quantum computing", search_type="keyword")

2. APPLICANT/COMPANY SEARCH (find patents by a company):
   results = ip_search.search_patent_globally("Apple", search_type="applicant")

3. INVENTOR SEARCH (find patents by a person):
   results = ip_search.search_patent_globally("Steve Jobs", search_type="inventor")

4. TRADEMARK SEARCH (find trademarks in specific regions):
   results = ip_search.search_trademarks("Nike", regions=["US", "International", "Korea"])

5. GENERATE REPORT (create a formatted table of all search links):
   report = ip_search.create_search_report("blockchain", "keyword")
   print(report)

All results return direct URLs to the respective patent office websites that you can:
- Copy and paste into a web browser
- Use for further research
- Export or save for documentation
"""

print(quick_guide)